In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
import gff3
import pandas as pd
import os
import statistics as stat
import random
import subprocess
import matplotlib.pyplot as plt
from scipy.stats import fisher_exact

In [42]:
### your fasta sequence file
fasta_file = '/Users/andreas/Streptomyces/S_coelicolor_A3_2_AL645882.2.fasta'
### you gff3 file
gff3_file = '/Users/andreas/Streptomyces/S_coelicolor_A3_2_AL645882.2_cleaned.gff3'
gff3 = pd.read_csv(gff3_file, sep = '\t', names = ['ID', 'source', 'feature', 'start', 'end', 'score', 'strand', 'phase', 'attributes'])

### output file name
output_file = '/Users/andreas/Streptomyces/TTA_genes.txt'

In [37]:
### load fasta file
with open(fasta_file, 'r') as f:
    lines = f.readlines()
    lines = lines[1:]
    lines = [l.replace('\n', '') for l in lines]
    lines = ''.join(lines)

### function to reverse complement genes on reverse strand
def reverse_complement(sequence):
    complement_dict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    reverse_seq = sequence[::-1]
    complement_seq = ''.join([complement_dict[base] for base in reverse_seq])
    return complement_seq

### Function to find check if DNA sequence contain a certain codon in the correct reading frame
def find_codon(DNA_sequence, codon):
    seq = Seq(DNA_sequence)
    codon_list = [DNA_sequence[i:i+3] for i in range(0, len(DNA_sequence), 3)]
    the_count = codon_list.count(codon)
    return the_count

In [38]:
### make dataframe with fasta sequences for all coding genes
fasta_sequences = []
CDS_df = pd.DataFrame()
for index, row in gff3.iterrows():
    if row['feature'] == 'CDS':
        if row['strand'] == '+':
            fasta_sequences.append(lines[row['start'] - 1:row['end']])
        else:
            fasta_sequences.append(reverse_complement(lines[row['start'] - 1:row['end']]))
        CDS_df = pd.concat([CDS_df, gff3[gff3.index == index]])
        # attributes.append(row['attributes'])
        # locus_tags.append(f"{row['start']}_{row['end']}_{row['strand']}")

CDS_df['fasta'] = fasta_sequences

In [39]:
### find genes with codon: TTA
TTA_genes = pd.DataFrame()
counts = []
for index, row in CDS_df.iterrows():
    if find_codon(row['fasta'], 'TTA') != 0:
        TTA_genes = pd.concat([TTA_genes, CDS_df[CDS_df.index == index]], ignore_index = True)
        counts.append(find_codon(row['fasta'], 'TTA'))
TTA_genes['n_TTA'] = counts

In [70]:
### extract gene names or gene synonoms from dataframe
def extract_names(attributes, id, exclude):
    thelist = []
    for att in attributes:
        att_list = att.split(';')
        x = 0
        for a in att_list:
            if a.find(id) == 0 and exclude not in a:
                thelist.append(a[len(id) + 1:])
                x += 1
                break
        if x == 0:
            thelist.append(f'no_{id}_name')
    return thelist

def extract_uni(attributes, id):
    thelist = []
    for att in attributes:
        n = att.find(id)
        if n != -1:
            thelist.append(att[n + len(id) + 1: n + len(id) + 7])
        else:
            thelist.append('no_uniprot_id')
    return thelist

TTA_genes['gene'] = extract_names(TTA_genes['attributes'], 'gene', 'gene_synonym')
TTA_genes['gene_synonym'] = gene_synonyms = extract_names(TTA_genes['attributes'], 'gene_synonym', '@')
TTA_genes['uniprot'] = extract_uni(TTA_genes['attributes'], 'UniProtKB/TrEMBL')


In [72]:
### save TTA genes as file
TTA_genes.to_csv(output_file, sep = '\t', header = True, index = False)

In [73]:
TTA_genes['uniprot'].to_csv(output_file + '_unitprot.txt', sep = '\t', header = False, index = False)